# 🤖 Deep Code Analysis Agent - Educational Notebook

**Tujuan:** Tutorial interaktif untuk membangun agen analisis kode berbasis AI menggunakan DeepAgents framework dengan FilesystemBackend.

**Yang akan dipelajari:**
- Konfigurasi AI model (ChatOpenAI) dengan temperature yang tepat
- Penggunaan FilesystemBackend untuk akses filesystem yang aman
- Pembuatan prompt sistem yang efektif untuk analisis kode
- Eksekusi agen dan pemrosesan hasil

**Prasyarat:**
- Python 3.10+
- Virtual environment aktif
- API key untuk LLM (OpenAI/Groq/etc)
- File `.env` dengan konfigurasi

**Referensi:**
- [DeepAgents Documentation](https://docs.deepagents.ai/)
- [LangChain OpenAI Integration](https://python.langchain.com/docs/integrations/chat/openai/)
- [FilesystemBackend Guide](https://python.langchain.com/docs/integrations/backends/filesystem/)

## 📦 Install Dependencies

Pastikan virtual environment aktif sebelum menjalankan cell ini.

In [ ]:
# Install required packages
%pip install deepagents langchain-openai python-dotenv pydantic

## 🔧 1. Import Required Libraries

Import library yang diperlukan dan load environment variables dari file `.env`. Ini adalah fondasi untuk script kita.

**Yang di-import:**
- `argparse`, `os`, `sys`, `time`: Utility Python standar
- `deepagents`: Framework untuk membuat AI agents
- `FilesystemBackend`: Backend built-in untuk akses filesystem
- `dotenv`: Load environment variables
- `ChatOpenAI`: LLM dari LangChain
- `SecretStr`: Untuk secure API key handling

**Referensi:**
- [DeepAgents Documentation](https://docs.deepagents.ai/)
- [LangChain OpenAI](https://python.langchain.com/docs/integrations/chat/openai/)

In [ ]:
import argparse
import os
import sys
import time

from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import SecretStr

# Load environment variables dari .env file
# Berisi: LITELLM_MODEL, LITELLM_VIRTUAL_KEY, LITELLM_API
# Optional: LANGSMITH_API_KEY, LANGSMITH_PROJECT untuk observability
load_dotenv()

print("✅ Libraries imported successfully")
print(f"🔧 Python version: {sys.version}")

## 🧠 2. Configure the AI Model

Setup language model yang akan menjadi otak dari agen kita. Model ini akan menggunakan built-in tools dari FilesystemBackend untuk menganalisis kode.

**Konsep penting:**
- **Model Selection**: Mendukung berbagai model via LiteLLM
- **Temperature Logic**: Reasoning models (GPT-5-mini) butuh temperature=1.0, model lain pakai 0.7
- **Security**: API key menggunakan `SecretStr` dari pydantic
- **Validation**: Pastikan environment variables tersedia

**Referensi:**
- [LangChain ChatOpenAI](https://python.langchain.com/docs/integrations/chat/openai/)
- [LiteLLM Models](https://docs.litellm.ai/docs/providers)

In [ ]:
# Ambil konfigurasi model dari environment atau default
model_name = os.getenv("LITELLM_MODEL", "gpt-4o-mini")
api_key = os.getenv("LITELLM_VIRTUAL_KEY")
api_base = os.getenv("LITELLM_API")

# Validasi environment variables yang required
if not api_key or not api_base:
    raise ValueError(
        "❌ Missing required environment variables:\n"
        "  LITELLM_VIRTUAL_KEY: LLM API key\n"
        "  LITELLM_API: LLM API base URL\n"
        "Please set these in your .env file or environment."
    )

# Tentukan apakah ini reasoning model (dengan "thinking" capabilities)
# Model seperti GPT-5-mini HANYA support temperature=1.0
# Model seperti gpt-oss-120b butuh specific temperature values
is_reasoning_model = any(
    keyword in model_name.lower()
    for keyword in ["gpt-5", "5-mini", "oss", "120b", "thinking", "reasoning"]
)

# Set temperature berdasarkan tipe model
# - Reasoning models: temperature=1.0 (required, lebih kreatif, explore lebih banyak options)
# - Model lain: temperature=0.7 (balanced, bagus untuk analysis)
# NOTE: azure/gpt-5-mini hanya support temperature=1.0, bukan 0.1
if is_reasoning_model:
    temperature = 1.0
else:
    temperature = 0.7  # Diubah dari 0.1 ke 0.7 untuk kompatibilitas model lebih luas

# Initialize ChatOpenAI model dengan konfigurasi kita
# Pakai SecretStr untuk API key agar satisfy type requirements
analysis_model = ChatOpenAI(
    api_key=SecretStr(api_key),
    model=model_name,
    base_url=api_base,
    temperature=temperature,
)

print("✅ AI Model configured successfully")
print(f"🤖 Model: {model_name}")
print(f"🌡️ Temperature: {temperature}")
print(f"🔗 API Base: {api_base}")

## 💾 3. Configure Filesystem Backend

LangChain's FilesystemBackend menyediakan built-in filesystem tools mengikuti BackendProtocol. Ini adalah alternatif best practice dari custom tools.

**Built-in tools yang disediakan otomatis:**
- `ls`: List files dengan metadata (size, modified_at, is_dir)
- `read_file`: Read file contents dengan offset/limit support untuk file besar
- `write_file`: Create new files dengan content validation
- `edit_file`: Perform exact string replacements di files
- `glob`: Advanced pattern matching dengan recursive support
- `grep`: Fast text search dengan ripgrep integration

**Keuntungan menggunakan built-in backend:**
- ✅ Tidak perlu custom tool definition (@tool decorators dieliminasi)
- ✅ Built-in security features (path validation, symlink protection)
- ✅ Automatic large content handling (eviction ke filesystem)
- ✅ LangGraph state integration via BackendProtocol
- ✅ Multiple backend support (FilesystemBackend, StateBackend, StoreBackend, dll)

**Referensi:**
- [LangChain FilesystemBackend](https://python.langchain.com/docs/integrations/backends/filesystem/)

## 📁 4. Setup Codebase Path

Setup path codebase yang akan dianalisis. Kita akan menggunakan path yang sudah ada di project ini.

**Fitur:**
- Default path ke Spring Boot demo project
- Fallback ke Casdoor project jika Spring Boot tidak ada
- Validation bahwa path exists dan merupakan directory
- Convert ke absolute path untuk consistency

In [ ]:
# Setup default codebase path
# Dalam notebook, kita akan menggunakan path yang sudah ada di project
default_codebase_path = "/Users/zeihanaulia/Programming/research/agent/dataset/codes/springboot-demo"

# Untuk notebook, kita langsung set codebase_path
# Dalam script asli menggunakan argparse, tapi di notebook kita simplify
codebase_path = os.getenv("CODEBASE_PATH", default_codebase_path)

# Dalam notebook environment, kita bisa interaktif prompt
# Tapi untuk demo, kita gunakan default
if not os.path.exists(codebase_path):
    print(f"⚠️  Default path tidak ada: {codebase_path}")
    print("Coba path lain yang valid...")
    # Fallback ke path yang ada
    codebase_path = "/Users/zeihanaulia/Programming/research/agent/dataset/codes/casdoor"
    if not os.path.exists(codebase_path):
        raise ValueError(f"❌ Codebase path tidak ditemukan: {codebase_path}")

# Validasi bahwa path adalah directory
if not os.path.isdir(codebase_path):
    raise ValueError(f"❌ Codebase path bukan directory: {codebase_path}")

# Convert ke absolute path untuk consistency
codebase_path = os.path.abspath(codebase_path)

print("✅ Codebase path configured")
print(f"📂 Target: {codebase_path}")

# List isi directory untuk verifikasi
try:
    import glob
    files = glob.glob(os.path.join(codebase_path, "*"))
    print(f"📄 Found {len(files)} items in directory")
    print("📋 Sample files:", [os.path.basename(f) for f in files[:5]])
except Exception as e:
    print(f"⚠️  Could not list directory: {e}")

## 📝 5. Create the Analysis Prompt

Buat system prompt yang akan mengarahkan agen dalam menganalisis codebase. Prompt ini menentukan WHAT dan HOW agen harus bekerja.

**Komponen prompt:**
- **Context**: Akses ke full codebase via built-in filesystem tools
- **Task**: Gather context → Identify purpose → Analyze code → Examine architecture → Summarize
- **Tools**: Dokumentasi built-in filesystem tools yang tersedia
- **Workflow**: Step-by-step guide untuk analysis
- **Best practices**: Tips menggunakan tools secara efektif

**Referensi:**
- [DeepAgents Prompting Guide](https://docs.deepagents.ai/prompting/)
- [LangChain Prompt Engineering](https://python.langchain.com/docs/guides/prompt_engineering/)

In [ ]:
# Buat system prompt yang comprehensive untuk analisis kode
analysis_prompt = f"""\
You are an expert code analysis agent. Your primary goal is to analyze the codebase and provide a comprehensive understanding of the project.

CODEBASE PATH: {codebase_path}

CONTEXT:
- You have access to the full codebase via built-in filesystem tools
- The workspace structure may be truncated, use tools to collect more context if needed
- Focus on gathering relevant context without going overboard

YOUR TASK:
1. **Gather Context**: Use ls and glob to explore the directory structure
2. **Identify Project Purpose**: Read README files, package.json, requirements.txt, pom.xml, or build.gradle to understand the project
3. **Analyze Code Content**: Read key source files to understand functionality
4. **Examine Architecture**: Map the project structure (folders, packages, layers)
5. **Summarize**: Provide a comprehensive overview with:
   - Project purpose and goals
   - Technology stack and dependencies
   - Architecture and main components
   - Key functionalities

BUILT-IN FILESYSTEM TOOLS (automatically available):
- ls(path): List files and directories with metadata (size, modified_at, is_dir)
- read_file(path, offset, limit): Read file contents with line numbers and pagination
- write_file(path, content): Create new files
- edit_file(path, old_string, new_string): Perform exact string replacements
- glob(pattern): Find files matching patterns (supports **/*.py recursive patterns)
- grep(pattern, path, glob): Fast text search with context

ANALYSIS WORKFLOW:
1. First, use ls or glob to understand the project layout
2. Read key configuration files (README.md, package.json, requirements.txt, setup.py, etc.)
3. Find and read main source files to understand the core functionality
4. Analyze the architecture based on your findings
5. Provide your comprehensive analysis with concrete examples from the code

TOOL USE BEST PRACTICES:
- Use glob() for pattern matching: glob("**/*.py"), glob("*.json"), glob("src/**/*.java")
- Use read_file() with offset/limit for pagination on large files
- Use grep() to search for specific patterns across files
- Combine tools in one response when possible to reduce turns

START EXPLORATION:
Begin by exploring the codebase structure and key files to build your understanding.
"""

print("✅ Analysis prompt created")
print(f"📏 Prompt length: {len(analysis_prompt)} characters")
print("🎯 Prompt preview:")
print(analysis_prompt[:200] + "...")

## 🤖 6. Instantiate Deep Agent with Filesystem Backend

Buat deep agent dengan FilesystemBackend untuk akses filesystem yang real dan aman. Backend menyediakan semua 6 filesystem tools secara otomatis.

**Yang terjadi di sini:**
- **FilesystemBackend**: Initialize dengan root_dir untuk security (agen hanya bisa akses files di bawah path ini)
- **create_deep_agent**: Buat agen dengan system prompt, model, dan backend
- **Automatic tools**: Agen otomatis dapat 6 built-in filesystem tools tanpa custom tool definitions

**Keamanan:** root_dir membatasi agen hanya bisa akses files dalam directory yang ditentukan.

**Referensi:**
- [DeepAgents create_deep_agent](https://docs.deepagents.ai/api/create_deep_agent/)
- [BackendProtocol](https://python.langchain.com/docs/concepts/architecture/#backend-protocol)

In [ ]:
# Configure FilesystemBackend dengan codebase path
# Ini memastikan agen hanya bisa akses files di bawah root_dir (security feature)
try:
    backend = FilesystemBackend(root_dir=codebase_path)
    print("✅ FilesystemBackend initialized")
    print(f"🔒 Root directory: {codebase_path}")
except Exception as e:
    raise RuntimeError(
        f"❌ Failed to initialize FilesystemBackend with root_dir={codebase_path}\n"
        f"Error: {str(e)}"
    ) from e

# Buat deep agent dengan backend
# Note: DeepAgents akan otomatis provide 6 built-in filesystem tools
try:
    agent = create_deep_agent(
        system_prompt=analysis_prompt,
        model=analysis_model,
        backend=backend,  # Pass backend instead of tools
    )
    print("✅ Deep agent created successfully")
    print("🛠️  Built-in tools available: ls, read_file, write_file, edit_file, glob, grep")
except Exception as e:
    raise RuntimeError(
        f"❌ Failed to create deep agent with FilesystemBackend\n"
        f"Error: {str(e)}"
    ) from e

## 🚀 7. Run the Agent and Analyze Codebase

Jalankan agen untuk menganalisis codebase. Ini adalah eksekusi utama dari workflow analisis.

**Yang terjadi:**
- **Display info**: Print codebase path, model, backend, temperature
- **Invoke agent**: Jalankan agen dengan input untuk menganalisis codebase
- **Handle errors**: Timeout dan exception handling
- **Measure time**: Track berapa lama analisis berjalan

**Agent workflow:**
1. Agen menggunakan tools untuk explore codebase
2. Membuat decisions tentang file mana yang perlu dibaca/dianalisis
3. Generate comprehensive analysis
4. Return semua messages dalam conversation history

In [ ]:
# Display startup information
print("=" * 80)
print("🤖 DEEP CODE ANALYSIS AGENT (NOTEBOOK MODE)")
print("=" * 80)
print(f"📁 Target Codebase: {codebase_path}")
print(f"🛠️  Model: {model_name}")
print("💾 Backend: FilesystemBackend (LangChain Built-in)")
print(f"🌡️  Temperature: {temperature}")
print("=" * 80)
print("🔍 Starting analysis... This may take a few moments.")
print()

# Track start time
start_time = time.time()

# Invoke agen dengan analysis task
# Agen akan:
# 1. Use tools untuk explore codebase
# 2. Make decisions tentang apa yang perlu dibaca/dianalisis
# 3. Generate comprehensive analysis
# 4. Return semua messages dalam conversation history

print(f"[{time.strftime('%H:%M:%S')}] 📋 Agent initialized with FilesystemBackend")
print(f"[{time.strftime('%H:%M:%S')}] 🔍 Starting codebase analysis...")

try:
    result = agent.invoke(
        {
            "input": f"Please analyze the codebase at {codebase_path}",
        },
        # Timeout untuk prevent infinite loops (dalam detik)
        # Note: Requires LangGraph version yang support timeout
    )
    analysis_time = time.time() - start_time
    print(f"[{time.strftime('%H:%M:%S')}] ✅ Analysis completed in {analysis_time:.2f} seconds")

except TimeoutError as e:
    analysis_time = time.time() - start_time
    print(f"❌ Agent analysis timed out after {analysis_time:.2f} seconds: {str(e)}")
    print("The agent took too long to complete the analysis.")
    result = None

except Exception as e:
    analysis_time = time.time() - start_time
    print(f"❌ Error during agent execution after {analysis_time:.2f} seconds: {str(e)}")
    import traceback
    traceback.print_exc()
    result = None

## 📊 8. Display Analysis Results

Tampilkan hasil analisis dari agen. Ini menampilkan comprehensive codebase analysis.

**Yang ditampilkan:**
- **Count tool calls**: Hitung berapa tool calls yang dibuat selama analysis
- **Extract final result**: Ambil AI message terakhir yang berisi comprehensive analysis
- **Display summary**: Tool calls, analysis time, statistics
- **Handle edge cases**: No results, failed executions

**Output:** Final analysis dengan project overview, technology stack, architecture, dan key functionalities.

In [ ]:
# Validate result structure
if not result or not isinstance(result, dict):
    print("❌ Error: Agent returned invalid result structure")
    print("Result:", result)
else:
    # Count tool calls yang dibuat selama analysis
    tool_call_counter = 0
    if "messages" in result:
        for msg in result["messages"]:
            if hasattr(msg, "tool_calls") and msg.tool_calls:
                tool_call_counter += len(msg.tool_calls)
    else:
        print("⚠️  Warning: No messages found in result")

    print("\n📈 Analysis Summary:")
    print(f"   • Tool calls made: {tool_call_counter}")
    print(f"   • Analysis time: {analysis_time:.2f} seconds")
    if tool_call_counter > 0:
        print(f"   • Average time per tool call: {analysis_time/tool_call_counter:.2f} seconds")
    else:
        print("   • No tool calls made (agent may have failed or been blocked)")

    print("\n" + "=" * 80)
    print("📊 FINAL ANALYSIS RESULT:")
    print("=" * 80)

    # Extract dan print hanya final result
    # AI message terakhir biasanya berisi comprehensive analysis
    if "messages" in result:
        final_messages = []
        for msg in result["messages"]:
            msg_type = type(msg).__name__
            content = getattr(msg, "content", None)
            has_content = content is not None and str(content).strip()
            has_tool_calls = hasattr(msg, "tool_calls") and msg.tool_calls

            # Kita mau AI messages yang berisi analysis (bukan tool calls)
            if has_content and not has_tool_calls and msg_type == "AIMessage":
                final_messages.append(str(content))

        # Show AI message terakhir (biasanya final analysis)
        if final_messages:
            print("FINAL RESULT:")
            print(final_messages[-1])
        else:
            print("❌ No detailed analysis result found.")
            if tool_call_counter == 0:
                print("\nPossible reasons:")
                print("  1. Agent failed to initialize properly")
                print("  2. Model API key or credentials are invalid")
                print("  3. Backend failed to provide filesystem tools")
                print("  4. Agent got stuck in infinite loop (check LangSmith trace)")
    else:
        print("❌ Error: No messages in result")

print("\n" + "=" * 80)
print("🎉 Analysis complete!")
print("=" * 80)

## 🔍 Optional: Detailed Message Trace

Secara opsional tampilkan semua messages yang di-exchange antara agen dan tools. Ini membantu memahami HOW agen bekerja.

**Yang ditampilkan:**
- **AI Messages**: Reasoning dan analysis dari agen
- **Tool Calls**: Function calls yang dibuat agen
- **Tool Responses**: Hasil eksekusi tools
- **Truncated outputs**: Long outputs dipotong untuk readability

**Tujuan:** Learning dan debugging - melihat decision-making process agen.

**Note:** Uncomment code di bawah untuk enable message trace.

In [ ]:
# Optional: Display detailed message trace (uncomment untuk enable)
# Ini membantu memahami HOW agen bekerja untuk sampai ke conclusion

if result and "messages" in result:
    print("\n" + "=" * 80)
    print("🔍 DETAILED MESSAGE TRACE (for learning/debugging):")
    print("=" * 80)

    for i, msg in enumerate(result["messages"]):
        msg_type = type(msg).__name__

        if hasattr(msg, "content") and msg.content:
            print(f"\n[{msg_type}]")
            print(msg.content)

        elif hasattr(msg, "tool_calls") and msg.tool_calls:
            print("\n[Tool Calls]")
            for call in msg.tool_calls:
                tool_name = call.get("name", "Unknown")
                args = call.get("args", {})
                print(
                    f"  → {tool_name}({', '.join(f'{k}={v}' for k, v in args.items())})" 
                )

        elif hasattr(msg, "name") and msg.name:
            print(f"\n[Tool Response: {msg.name}]")
            content = getattr(msg, "content", "No content")
            # Truncate long outputs untuk readability
            if len(str(content)) > 500:
                print(str(content)[:500] + "\n... (truncated)")
            else:
                print(content)

    print("\n" + "=" * 80)
else:
    print("\n⚠️  No message trace available (agent may have failed)")

## 🎓 Summary & Learning Outcomes

**Yang telah kita pelajari:**

### 🔑 Key Concepts
1. **FilesystemBackend**: LangChain's built-in filesystem abstraction menggantikan custom tools
2. **BackendProtocol**: Interface standar untuk berbagai backend types
3. **Built-in Tools**: 6 automatic filesystem tools (ls, read_file, write_file, edit_file, glob, grep)
4. **Security**: root_dir sandboxing mencegah akses unauthorized
5. **Temperature Logic**: Reasoning models vs regular models memiliki requirements berbeda

### 🛠️ Technical Skills
- Setup AI model dengan proper configuration
- Create comprehensive system prompts untuk code analysis
- Initialize dan configure FilesystemBackend
- Handle agent execution dan error cases
- Process dan display agent results

### 📚 Best Practices
- Use built-in backends instead of custom tools when possible
- Validate environment variables dan paths
- Handle timeouts dan exceptions gracefully
- Provide clear user feedback dan progress indicators
- Structure prompts dengan clear workflow steps

### 🔗 Next Steps
- Experiment dengan codebase berbeda
- Coba model LLM yang berbeda
- Explore StateBackend atau StoreBackend untuk use cases lain
- Add custom tools jika built-in tools tidak cukup
- Integrate dengan observability tools seperti LangSmith

**Referensi lengkap:**
- [DeepAgents Framework](https://docs.deepagents.ai/)
- [LangChain Documentation](https://python.langchain.com/docs/)
- [FilesystemBackend Guide](https://python.langchain.com/docs/integrations/backends/filesystem/)